In [1]:
import mlflow 

In [2]:
print("hello")

hello


In [4]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [5]:
import pandas

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler    

# infer signature -  important for inferencing the model

In [8]:
from mlflow.models import infer_signature

In [ ]:
## set tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
from sklearn.datasets import load_wine

In [11]:
X,y = datasets.load_wine(return_X_y=True)

# Splitting data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Model hyperparameters in Randomforestclassifier

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
params = {
    "n_estimators": 50,
    "criterion": 'gini',
    "max_depth": 5,
    "random_state": 42,
    "n_jobs": -1
}

In [18]:
df = load_wine(as_frame=True)

In [51]:
df

{'data':      alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
 0      14.23        1.71  2.43               15.6      127.0           2.80   
 1      13.20        1.78  2.14               11.2      100.0           2.65   
 2      13.16        2.36  2.67               18.6      101.0           2.80   
 3      14.37        1.95  2.50               16.8      113.0           3.85   
 4      13.24        2.59  2.87               21.0      118.0           2.80   
 ..       ...         ...   ...                ...        ...            ...   
 173    13.71        5.65  2.45               20.5       95.0           1.68   
 174    13.40        3.91  2.48               23.0      102.0           1.80   
 175    13.27        4.28  2.26               20.0      120.0           1.59   
 176    13.17        2.59  2.37               20.0      120.0           1.65   
 177    14.13        4.10  2.74               24.5       96.0           2.05   
 
      flavanoids  nonflavanoid

In [50]:
X.shape

(178, 13)

In [37]:
rf = RandomForestClassifier(**params)
rf.fit(X_train, y_train)    

RandomForestClassifier(max_depth=5, n_estimators=50, n_jobs=-1, random_state=42)

In [38]:
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [39]:
y_pred

array([0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 2, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 1, 1,
       2])

In [40]:
acc=accuracy_score(y_test, y_pred)


In [41]:
print(acc)

1.0


In [42]:
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

[[15  0  0]
 [ 0 18  0]
 [ 0  0 12]]


## MLFLow Tracking

In [43]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

*create a new experiment*

In [44]:
mlflow.set_experiment("wine_rf_experiment")

with mlflow.start_run():
    #log the hyperparameters
    mlflow.log_params(params)
    
    #log accuracy metrics
    mlflow.log_metric("accuracy", acc) # Convert to list for logging
    
    #Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Training", "Using a model = RandomForestClassifier")
    
    ## Infer the model signature
    signature = infer_signature(X_train, rf.predict(X_train))
    
    #log the model
    model_info = mlflow.sklearn.log_model(
        rf,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        registered_model_name="WineQualityRandomForestModel"
    )

Registered model 'WineQualityRandomForestModel' already exists. Creating a new version of this model...
2025/05/31 15:24:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WineQualityRandomForestModel, version 3


🏃 View run popular-grouse-937 at: http://127.0.0.1:5000/#/experiments/545408099770630109/runs/692bed3b400c4e258178e4d220db088f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/545408099770630109


Created version '3' of model 'WineQualityRandomForestModel'.


In [45]:
model_info.model_uri

'runs:/692bed3b400c4e258178e4d220db088f/model'

# Method 1:

# Inferencing and Validating before Deployment

In [52]:
model_uri

'runs:/692bed3b400c4e258178e4d220db088f/model'

In [55]:
import mlflow
from mlflow.models import validate_serving_input


model_uri = 'runs:/692bed3b400c4e258178e4d220db088f/model'
# The model is logged with an input example
serving_payload = """ {
    "inputs": [
  [
    13.16,
    2.36,
    2.67,
    18.6,
    101,
    2.8,
    3.24,
    0.3,
    2.81,
    5.68,
    1.03,
    3.17,
    1185
  ],
  [
    12.08,
    2.08,
    1.7,
    17.5,
    97,
    2.23,
    2.17,
    0.26,
    1.4,
    3.3,
    1.27,
    2.96,
    710
  ],
  [
    12.42,
    4.43,
    2.73,
    26.5,
    102,
    2.2,
    2.13,
    0.43,
    1.71,
    2.08,
    0.92,
    3.12,
    365
  ],
  [
    12.58,
    1.29,
    2.1,
    20,
    103,
    1.48,
    0.58,
    0.53,
    1.4,
    7.6,
    0.58,
    1.55,
    640
  ],
  [
    13.83,
    1.65,
    2.6,
    17.2,
    94,
    2.45,
    2.99,
    0.22,
    2.29,
    5.6,
    1.24,
    3.37,
    1265
  ],
  [
    13.03,
    0.9,
    1.71,
    16,
    86,
    1.95,
    2.03,
    0.24,
    1.46,
    4.6,
    1.19,
    2.48,
    392
  ],
  [
    14.22,
    1.7,
    2.3,
    16.3,
    118,
    3.2,
    3,
    0.26,
    2.03,
    6.38,
    0.94,
    3.31,
    970
  ],
  [
    13.39,
    1.77,
    2.62,
    16.1,
    93,
    2.85,
    2.94,
    0.34,
    1.45,
    4.8,
    0.92,
    3.22,
    1195
  ],
  [
    13.23,
    3.3,
    2.28,
    18.5,
    98,
    1.8,
    0.83,
    0.61,
    1.87,
    10.52,
    0.56,
    1.51,
    675
  ],
  [
    13.49,
    3.59,
    2.19,
    19.5,
    88,
    1.62,
    0.48,
    0.58,
    0.88,
    5.7,
    0.81,
    1.82,
    580
  ],
  [
    12.51,
    1.73,
    1.98,
    20.5,
    85,
    2.2,
    1.92,
    0.32,
    1.48,
    2.94,
    1.04,
    3.57,
    672
  ],
  [
    12.33,
    0.99,
    1.95,
    14.8,
    136,
    1.9,
    1.85,
    0.35,
    2.76,
    3.4,
    1.06,
    2.31,
    750
  ],
  [
    13.28,
    1.64,
    2.84,
    15.5,
    110,
    2.6,
    2.68,
    0.34,
    1.36,
    4.6,
    1.09,
    2.78,
    880
  ],
  [
    12.29,
    2.83,
    2.22,
    18,
    88,
    2.45,
    2.25,
    0.25,
    1.99,
    2.15,
    1.15,
    3.3,
    290
  ],
  [
    13.71,
    1.86,
    2.36,
    16.6,
    101,
    2.61,
    2.88,
    0.27,
    1.69,
    3.8,
    1.11,
    4,
    1035
  ],
  [
    13.88,
    5.04,
    2.23,
    20,
    80,
    0.98,
    0.34,
    0.4,
    0.68,
    4.9,
    0.58,
    1.33,
    415
  ],
  [
    12.29,
    1.41,
    1.98,
    16,
    85,
    2.55,
    2.5,
    0.29,
    1.77,
    2.9,
    1.23,
    2.74,
    428
  ],
  [
    12.21,
    1.19,
    1.75,
    16.8,
    151,
    1.85,
    1.28,
    0.14,
    2.5,
    2.85,
    1.28,
    3.07,
    718
  ],
  [
    12.82,
    3.37,
    2.3,
    19.5,
    88,
    1.48,
    0.66,
    0.4,
    0.97,
    10.26,
    0.72,
    1.75,
    685
  ],
  [
    14.12,
    1.48,
    2.32,
    16.8,
    95,
    2.2,
    2.43,
    0.26,
    1.57,
    5,
    1.17,
    2.82,
    1280
  ],
  [
    14.39,
    1.87,
    2.45,
    14.6,
    96,
    2.5,
    2.52,
    0.3,
    1.98,
    5.25,
    1.02,
    3.58,
    1290
  ],
  [
    13.3,
    1.72,
    2.14,
    17,
    94,
    2.4,
    2.19,
    0.27,
    1.35,
    3.95,
    1.02,
    2.77,
    1285
  ],
  [
    12.25,
    3.88,
    2.2,
    18.5,
    112,
    1.38,
    0.78,
    0.29,
    1.14,
    8.21,
    0.65,
    2,
    855
  ],
  [
    13.24,
    2.59,
    2.87,
    21,
    118,
    2.8,
    2.69,
    0.39,
    1.82,
    4.32,
    1.04,
    2.93,
    735
  ],
  [
    13.68,
    1.83,
    2.36,
    17.2,
    104,
    2.42,
    2.69,
    0.42,
    1.97,
    3.84,
    1.23,
    2.87,
    990
  ],
  [
    12.47,
    1.52,
    2.2,
    19,
    162,
    2.5,
    2.27,
    0.32,
    3.28,
    2.6,
    1.16,
    2.63,
    937
  ],
  [
    12.2,
    3.03,
    2.32,
    19,
    96,
    1.25,
    0.49,
    0.4,
    0.73,
    5.5,
    0.66,
    1.83,
    510
  ],
  [
    11.66,
    1.88,
    1.92,
    16,
    97,
    1.61,
    1.57,
    0.34,
    1.15,
    3.8,
    1.23,
    2.14,
    428
  ],
  [
    14.1,
    2.16,
    2.3,
    18,
    105,
    2.95,
    3.32,
    0.22,
    2.38,
    5.75,
    1.25,
    3.17,
    1510
  ],
  [
    12.87,
    4.61,
    2.48,
    21.5,
    86,
    1.7,
    0.65,
    0.47,
    0.86,
    7.65,
    0.54,
    1.86,
    625
  ],
  [
    13.05,
    5.8,
    2.13,
    21.5,
    86,
    2.62,
    2.65,
    0.3,
    2.01,
    2.6,
    0.73,
    3.1,
    380
  ],
  [
    14.23,
    1.71,
    2.43,
    15.6,
    127,
    2.8,
    3.06,
    0.28,
    2.29,
    5.64,
    1.04,
    3.92,
    1065
  ],
  [
    13.52,
    3.17,
    2.72,
    23.5,
    97,
    1.55,
    0.52,
    0.5,
    0.55,
    4.35,
    0.89,
    2.06,
    520
  ],
  [
    12.43,
    1.53,
    2.29,
    21.5,
    86,
    2.74,
    3.15,
    0.39,
    1.77,
    3.94,
    0.69,
    2.84,
    352
  ],
  [
    12.17,
    1.45,
    2.53,
    19,
    104,
    1.89,
    1.75,
    0.45,
    1.03,
    2.95,
    1.45,
    2.23,
    355
  ],
  [
    13.05,
    1.77,
    2.1,
    17,
    107,
    3,
    3,
    0.28,
    2.03,
    5.04,
    0.88,
    3.35,
    885
  ],
  [
    11.81,
    2.12,
    2.74,
    21.5,
    134,
    1.6,
    0.99,
    0.14,
    1.56,
    2.5,
    0.95,
    2.26,
    625
  ],
  [
    13.87,
    1.9,
    2.8,
    19.4,
    107,
    2.95,
    2.97,
    0.37,
    1.76,
    4.5,
    1.25,
    3.4,
    915
  ],
  [
    13.56,
    1.71,
    2.31,
    16.2,
    117,
    3.15,
    3.29,
    0.34,
    2.34,
    6.13,
    0.95,
    3.38,
    795
  ],
  [
    11.79,
    2.13,
    2.78,
    28.5,
    92,
    2.13,
    2.24,
    0.58,
    1.76,
    3,
    0.97,
    2.44,
    466
  ],
  [
    13.05,
    2.05,
    3.22,
    25,
    124,
    2.63,
    2.68,
    0.47,
    1.92,
    3.58,
    1.13,
    3.2,
    830
  ],
  [
    12.85,
    1.6,
    2.52,
    17.8,
    95,
    2.48,
    2.37,
    0.26,
    1.46,
    3.93,
    1.09,
    3.63,
    1015
  ],
  [
    12.81,
    2.31,
    2.4,
    24,
    98,
    1.15,
    1.09,
    0.27,
    0.83,
    5.7,
    0.66,
    1.36,
    560
  ],
  [
    12.72,
    1.81,
    2.2,
    18.8,
    86,
    2.2,
    2.53,
    0.26,
    1.77,
    3.9,
    1.16,
    3.14,
    714
  ],
  [
    12.7,
    3.87,
    2.4,
    23,
    101,
    2.83,
    2.55,
    0.43,
    1.95,
    2.57,
    1.19,
    3.13,
    463
  ],
  [
    11.84,
    0.89,
    2.58,
    18,
    94,
    2.2,
    2.21,
    0.22,
    2.35,
    3.05,
    0.79,
    3.08,
    520
  ],
  [
    14.22,
    3.99,
    2.51,
    13.2,
    128,
    3,
    3.04,
    0.2,
    2.08,
    5.1,
    0.89,
    3.53,
    760
  ],
  [
    12.6,
    1.34,
    1.9,
    18.5,
    88,
    1.45,
    1.36,
    0.29,
    1.35,
    2.45,
    1.04,
    2.77,
    562
  ],
  [
    12.16,
    1.61,
    2.31,
    22.8,
    90,
    1.78,
    1.69,
    0.43,
    1.56,
    2.45,
    1.33,
    2.26,
    495
  ],
  [
    11.45,
    2.4,
    2.42,
    20,
    96,
    2.9,
    2.79,
    0.32,
    1.83,
    3.25,
    0.8,
    3.39,
    625
  ],
  [
    13.71,
    5.65,
    2.45,
    20.5,
    95,
    1.68,
    0.61,
    0.52,
    1.06,
    7.7,
    0.64,
    1.74,
    740
  ],
  [
    12.85,
    3.27,
    2.58,
    22,
    106,
    1.65,
    0.6,
    0.6,
    0.96,
    5.58,
    0.87,
    2.11,
    570
  ],
  [
    13.9,
    1.68,
    2.12,
    16,
    101,
    3.1,
    3.39,
    0.21,
    2.14,
    6.1,
    0.91,
    3.33,
    985
  ],
  [
    11.62,
    1.99,
    2.28,
    18,
    98,
    3.02,
    2.26,
    0.17,
    1.35,
    3.25,
    1.16,
    2.96,
    345
  ],
  [
    12.6,
    2.46,
    2.2,
    18.5,
    94,
    1.62,
    0.66,
    0.63,
    0.94,
    7.1,
    0.73,
    1.58,
    695
  ],
  [
    13.45,
    3.7,
    2.6,
    23,
    111,
    1.7,
    0.92,
    0.43,
    1.46,
    10.68,
    0.85,
    1.56,
    695
  ],
  [
    12.64,
    1.36,
    2.02,
    16.8,
    100,
    2.02,
    1.41,
    0.53,
    0.62,
    5.75,
    0.98,
    1.59,
    450
  ],
  [
    12.99,
    1.67,
    2.6,
    30,
    139,
    3.3,
    2.89,
    0.21,
    1.96,
    3.35,
    1.31,
    3.5,
    985
  ],
  [
    13.76,
    1.53,
    2.7,
    19.5,
    132,
    2.95,
    2.74,
    0.5,
    1.35,
    5.4,
    1.25,
    3,
    1235
  ],
  [
    12.07,
    2.16,
    2.17,
    21,
    85,
    2.6,
    2.65,
    0.37,
    1.35,
    2.76,
    0.86,
    3.28,
    378
  ],
  [
    12.7,
    3.55,
    2.36,
    21.5,
    106,
    1.7,
    1.2,
    0.17,
    0.84,
    5,
    0.78,
    1.29,
    600
  ],
  [
    12.84,
    2.96,
    2.61,
    24,
    101,
    2.32,
    0.6,
    0.53,
    0.81,
    4.92,
    0.89,
    2.15,
    590
  ],
  [
    13.67,
    1.25,
    1.92,
    18,
    94,
    2.1,
    1.79,
    0.32,
    0.73,
    3.8,
    1.23,
    2.46,
    630
  ],
  [
    13.17,
    2.59,
    2.37,
    20,
    120,
    1.65,
    0.68,
    0.53,
    1.46,
    9.3,
    0.6,
    1.62,
    840
  ],
  [
    11.76,
    2.68,
    2.92,
    20,
    103,
    1.75,
    2.03,
    0.6,
    1.05,
    3.8,
    1.23,
    2.5,
    607
  ],
  [
    11.03,
    1.51,
    2.2,
    21.5,
    85,
    2.46,
    2.17,
    0.52,
    2.01,
    1.9,
    1.71,
    2.87,
    407
  ],
  [
    12.42,
    2.55,
    2.27,
    22,
    90,
    1.68,
    1.84,
    0.66,
    1.42,
    2.7,
    0.86,
    3.3,
    315
  ],
  [
    13.77,
    1.9,
    2.68,
    17.1,
    115,
    3,
    2.79,
    0.39,
    1.68,
    6.3,
    1.13,
    2.93,
    1375
  ],
  [
    14.2,
    1.76,
    2.45,
    15.2,
    112,
    3.27,
    3.39,
    0.34,
    1.97,
    6.75,
    1.05,
    2.85,
    1450
  ],
  [
    12.36,
    3.83,
    2.38,
    21,
    88,
    2.3,
    0.92,
    0.5,
    1.04,
    7.65,
    0.56,
    1.58,
    520
  ],
  [
    14.37,
    1.95,
    2.5,
    16.8,
    113,
    3.85,
    3.49,
    0.24,
    2.18,
    7.8,
    0.86,
    3.45,
    1480
  ],
  [
    13.73,
    4.36,
    2.26,
    22.5,
    88,
    1.28,
    0.47,
    0.52,
    1.15,
    6.62,
    0.78,
    1.75,
    520
  ],
  [
    13.94,
    1.73,
    2.27,
    17.4,
    108,
    2.88,
    3.54,
    0.32,
    2.08,
    8.9,
    1.12,
    3.1,
    1260
  ],
  [
    13.48,
    1.81,
    2.41,
    20.5,
    100,
    2.7,
    2.98,
    0.26,
    1.86,
    5.1,
    1.04,
    3.47,
    920
  ],
  [
    12,
    0.92,
    2,
    19,
    86,
    2.42,
    2.26,
    0.3,
    1.43,
    2.5,
    1.38,
    3.12,
    278
  ],
  [
    11.84,
    2.89,
    2.23,
    18,
    112,
    1.72,
    1.32,
    0.43,
    0.95,
    2.65,
    0.96,
    2.52,
    500
  ],
  [
    13.51,
    1.8,
    2.65,
    19,
    110,
    2.35,
    2.53,
    0.29,
    1.54,
    4.2,
    1.1,
    2.87,
    1095
  ],
  [
    14.38,
    3.59,
    2.28,
    16,
    102,
    3.25,
    3.17,
    0.27,
    2.19,
    4.9,
    1.04,
    3.44,
    1065
  ],
  [
    14.06,
    2.15,
    2.61,
    17.6,
    121,
    2.6,
    2.51,
    0.31,
    1.25,
    5.05,
    1.06,
    3.58,
    1295
  ],
  [
    11.87,
    4.31,
    2.39,
    21,
    82,
    2.86,
    3.03,
    0.21,
    2.91,
    2.8,
    0.75,
    3.64,
    380
  ],
  [
    13.24,
    3.98,
    2.29,
    17.5,
    103,
    2.64,
    2.63,
    0.32,
    1.66,
    4.36,
    0.82,
    3,
    680
  ],
  [
    12.29,
    1.61,
    2.21,
    20.4,
    103,
    1.1,
    1.02,
    0.37,
    1.46,
    3.05,
    0.906,
    1.82,
    870
  ],
  [
    12.25,
    4.72,
    2.54,
    21,
    89,
    1.38,
    0.47,
    0.53,
    0.8,
    3.85,
    0.75,
    1.27,
    720
  ],
  [
    11.46,
    3.74,
    1.82,
    19.5,
    107,
    3.18,
    2.58,
    0.24,
    3.58,
    2.9,
    0.75,
    2.81,
    562
  ],
  [
    12,
    1.51,
    2.42,
    22,
    86,
    1.45,
    1.25,
    0.5,
    1.63,
    3.6,
    1.05,
    2.65,
    450
  ],
  [
    13.05,
    3.86,
    2.32,
    22.5,
    85,
    1.65,
    1.59,
    0.61,
    1.62,
    4.8,
    0.84,
    2.01,
    515
  ],
  [
    13.84,
    4.12,
    2.38,
    19.5,
    89,
    1.8,
    0.83,
    0.48,
    1.56,
    9.01,
    0.57,
    1.64,
    480
  ],
  [
    13.69,
    3.26,
    2.54,
    20,
    107,
    1.83,
    0.56,
    0.5,
    0.8,
    5.88,
    0.96,
    1.82,
    680
  ],
  [
    12.08,
    1.33,
    2.3,
    23.6,
    70,
    2.2,
    1.59,
    0.42,
    1.38,
    1.74,
    1.07,
    3.21,
    625
  ],
  [
    14.83,
    1.64,
    2.17,
    14,
    97,
    2.8,
    2.98,
    0.29,
    1.98,
    5.2,
    1.08,
    2.85,
    1045
  ],
  [
    14.75,
    1.73,
    2.39,
    11.4,
    91,
    3.1,
    3.69,
    0.43,
    2.81,
    5.4,
    1.25,
    2.73,
    1150
  ],
  [
    12.37,
    0.94,
    1.36,
    10.6,
    88,
    1.98,
    0.57,
    0.28,
    0.42,
    1.95,
    1.05,
    1.82,
    520
  ],
  [
    13.11,
    1.9,
    2.75,
    25.5,
    116,
    2.2,
    1.28,
    0.26,
    1.56,
    7.1,
    0.61,
    1.33,
    425
  ],
  [
    12.88,
    2.99,
    2.4,
    20,
    104,
    1.3,
    1.22,
    0.24,
    0.83,
    5.4,
    0.74,
    1.42,
    530
  ],
  [
    13.83,
    1.57,
    2.62,
    20,
    115,
    2.95,
    3.4,
    0.4,
    1.72,
    6.6,
    1.13,
    2.57,
    1130
  ],
  [
    13.49,
    1.66,
    2.24,
    24,
    87,
    1.88,
    1.84,
    0.27,
    1.03,
    3.74,
    0.98,
    2.78,
    472
  ],
  [
    13.17,
    5.19,
    2.32,
    22,
    93,
    1.74,
    0.63,
    0.61,
    1.55,
    7.9,
    0.6,
    1.48,
    725
  ],
  [
    12.51,
    1.24,
    2.25,
    17.5,
    85,
    2,
    0.58,
    0.6,
    1.25,
    5.45,
    0.75,
    1.51,
    650
  ],
  [
    13.32,
    3.24,
    2.38,
    21.5,
    92,
    1.93,
    0.76,
    0.45,
    1.25,
    8.42,
    0.55,
    1.62,
    650
  ],
  [
    12.96,
    3.45,
    2.35,
    18.5,
    106,
    1.39,
    0.7,
    0.4,
    0.94,
    5.28,
    0.68,
    1.75,
    675
  ],
  [
    12.37,
    1.13,
    2.16,
    19,
    87,
    3.5,
    3.1,
    0.19,
    1.87,
    4.45,
    1.22,
    2.87,
    420
  ],
  [
    13.74,
    1.67,
    2.25,
    16.4,
    118,
    2.6,
    2.9,
    0.21,
    1.62,
    5.85,
    0.92,
    3.2,
    1060
  ],
  [
    12.72,
    1.75,
    2.28,
    22.5,
    84,
    1.38,
    1.76,
    0.48,
    1.63,
    3.3,
    0.88,
    2.42,
    488
  ],
  [
    13.05,
    1.73,
    2.04,
    12.4,
    92,
    2.72,
    3.27,
    0.17,
    2.91,
    7.2,
    1.12,
    2.91,
    1150
  ],
  [
    13.27,
    4.28,
    2.26,
    20,
    120,
    1.59,
    0.69,
    0.43,
    1.35,
    10.2,
    0.59,
    1.56,
    835
  ],
  [
    13.72,
    1.43,
    2.5,
    16.7,
    108,
    3.4,
    3.67,
    0.19,
    2.04,
    6.8,
    0.89,
    2.87,
    1285
  ],
  [
    14.1,
    2.02,
    2.4,
    18.8,
    103,
    2.75,
    2.92,
    0.32,
    2.38,
    6.2,
    1.07,
    2.75,
    1060
  ],
  [
    11.64,
    2.06,
    2.46,
    21.6,
    84,
    1.95,
    1.69,
    0.48,
    1.35,
    2.8,
    1,
    2.75,
    680
  ],
  [
    12.93,
    3.8,
    2.65,
    18.6,
    102,
    2.41,
    2.41,
    0.25,
    1.98,
    4.5,
    1.03,
    3.52,
    770
  ],
  [
    13.29,
    1.97,
    2.68,
    16.8,
    102,
    3,
    3.23,
    0.31,
    1.66,
    6,
    1.07,
    2.84,
    1270
  ],
  [
    14.16,
    2.51,
    2.48,
    20,
    91,
    1.68,
    0.7,
    0.44,
    1.24,
    9.7,
    0.62,
    1.71,
    660
  ],
  [
    12.04,
    4.3,
    2.38,
    22,
    80,
    2.1,
    1.75,
    0.42,
    1.35,
    2.6,
    0.79,
    2.57,
    580
  ],
  [
    13.05,
    1.65,
    2.55,
    18,
    98,
    2.45,
    2.43,
    0.29,
    1.44,
    4.25,
    1.12,
    2.51,
    1105
  ],
  [
    12.45,
    3.03,
    2.64,
    27,
    97,
    1.9,
    0.58,
    0.63,
    1.14,
    7.5,
    0.67,
    1.73,
    880
  ],
  [
    13.58,
    2.58,
    2.69,
    24.5,
    105,
    1.55,
    0.84,
    0.39,
    1.54,
    8.66,
    0.74,
    1.8,
    750
  ],
  [
    13.2,
    1.78,
    2.14,
    11.2,
    100,
    2.65,
    2.76,
    0.26,
    1.28,
    4.38,
    1.05,
    3.4,
    1050
  ],
  [
    13.82,
    1.75,
    2.42,
    14,
    111,
    3.88,
    3.74,
    0.32,
    1.87,
    7.05,
    1.01,
    3.26,
    1190
  ],
  [
    13.08,
    3.9,
    2.36,
    21.5,
    113,
    1.41,
    1.39,
    0.34,
    1.14,
    9.4,
    0.57,
    1.33,
    550
  ],
  [
    12.86,
    1.35,
    2.32,
    18,
    122,
    1.51,
    1.25,
    0.21,
    0.94,
    4.1,
    0.76,
    1.29,
    630
  ],
  [
    12.79,
    2.67,
    2.48,
    22,
    112,
    1.48,
    1.36,
    0.24,
    1.26,
    10.8,
    0.48,
    1.47,
    480
  ],
  [
    11.82,
    1.72,
    1.88,
    19.5,
    86,
    2.5,
    1.64,
    0.37,
    1.42,
    2.06,
    0.94,
    2.44,
    415
  ],
  [
    12.29,
    3.17,
    2.21,
    18,
    88,
    2.85,
    2.99,
    0.45,
    2.81,
    2.3,
    1.42,
    2.83,
    406
  ],
  [
    11.82,
    1.47,
    1.99,
    20.8,
    86,
    1.98,
    1.6,
    0.3,
    1.53,
    1.95,
    0.95,
    3.33,
    495
  ],
  [
    11.65,
    1.67,
    2.62,
    26,
    88,
    1.92,
    1.61,
    0.4,
    1.34,
    2.6,
    1.36,
    3.21,
    562
  ],
  [
    11.96,
    1.09,
    2.3,
    21,
    101,
    3.38,
    2.14,
    0.13,
    1.65,
    3.21,
    0.99,
    3.13,
    886
  ],
  [
    11.56,
    2.05,
    3.23,
    28.5,
    119,
    3.18,
    5.08,
    0.47,
    1.87,
    6,
    0.93,
    3.69,
    465
  ],
  [
    14.13,
    4.1,
    2.74,
    24.5,
    96,
    2.05,
    0.76,
    0.56,
    1.35,
    9.2,
    0.61,
    1.6,
    560
  ],
  [
    14.06,
    1.63,
    2.28,
    16,
    126,
    3,
    3.17,
    0.24,
    2.1,
    5.65,
    1.09,
    3.71,
    780
  ],
  [
    13.86,
    1.51,
    2.67,
    25,
    86,
    2.95,
    2.86,
    0.21,
    1.87,
    3.38,
    1.36,
    3.16,
    410
  ],
  [
    12.25,
    1.73,
    2.12,
    19,
    80,
    1.65,
    2.03,
    0.37,
    1.63,
    3.4,
    1,
    3.17,
    510
  ],
  [
    14.38,
    1.87,
    2.38,
    12,
    102,
    3.3,
    3.64,
    0.29,
    2.96,
    7.5,
    1.2,
    3,
    1547
  ],
  [
    12.69,
    1.53,
    2.26,
    20.7,
    80,
    1.38,
    1.46,
    0.58,
    1.62,
    3.05,
    0.96,
    2.06,
    495
  ],
  [
    12.34,
    2.45,
    2.46,
    21,
    98,
    2.56,
    2.11,
    0.34,
    1.31,
    2.8,
    0.8,
    3.38,
    438
  ]
]
    
    
}"""

validate_serving_input(model_uri,serving_payload)

array([0, 1, 1, 2, 0, 1, 0, 0, 2, 2, 1, 1, 0, 1, 0, 2, 1, 1, 2, 0, 0, 0,
       2, 0, 0, 1, 2, 1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 2, 1,
       1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 2, 2, 1, 1, 0, 1, 2, 2, 1, 2, 1, 1,
       1, 0, 0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 2,
       1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0,
       2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0, 1,
       1])

# Method 2:

# load model back for prediction as a generic python function model

In [57]:
import pandas as pd

In [58]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

wine_features_name = datasets.load_wine().feature_names
result = pd.DataFrame(X_test,columns=wine_features_name)
result['actual_class'] = y_test
result['predicted_class'] = predictions

In [59]:
result

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,actual_class,predicted_class
0,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.100000,0.96,3.36,845.0,0,0
1,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.240000,0.87,3.33,1080.0,0,0
2,12.93,2.81,2.70,21.0,96.0,1.54,0.50,0.53,0.75,4.600000,0.77,2.31,600.0,2,2
3,13.73,1.50,2.70,22.5,101.0,3.00,3.25,0.29,2.38,5.700000,1.19,2.71,1285.0,0,0
4,12.37,1.17,1.92,19.6,78.0,2.11,2.00,0.27,1.04,4.680000,1.12,3.48,510.0,1,1
5,14.30,1.92,2.72,20.0,120.0,2.80,3.14,0.33,1.97,6.200000,1.07,2.65,1280.0,0,0
6,12.00,3.43,2.00,19.0,87.0,2.00,1.64,0.37,1.87,1.280000,0.93,3.05,564.0,1,1
7,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.300000,0.70,1.56,750.0,2,2
8,11.61,1.35,2.70,20.0,94.0,2.74,2.92,0.29,2.49,2.650000,0.96,3.26,680.0,1,1
9,13.36,2.56,2.35,20.0,89.0,1.40,0.50,0.37,0.64,5.600000,0.70,2.47,780.0,2,2


# Model Registry

In [61]:
mlflow.set_experiment("wine_rf_experiment")

with mlflow.start_run():
    #log the hyperparameters
    mlflow.log_params(params)
    
    #log accuracy metrics
    mlflow.log_metric("accuracy", 1.0) # Convert to list for logging
    
    #Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Model Training", "Using a model = RandomForestClassifier")
    
    ## Infer the model signature
    signature = infer_signature(X_train, rf.predict(X_train))
    
    #log the model
    model_info = mlflow.sklearn.log_model(
        rf,
        artifact_path="model",
        signature=signature,
        input_example=X_train,
        #registered_model_name="WineQualityRandomForestModel"
    )

🏃 View run valuable-ape-972 at: http://127.0.0.1:5000/#/experiments/545408099770630109/runs/d8c5179bb11a489eaf24b3a0785322d1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/545408099770630109


## Inferencing from model version from registry

In [67]:
import mlflow.sklearn
model_name = "WineQualityRandomForestModel"
model_version = 5

model_uri  = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)


In [68]:
model_uri

'models:/WineQualityRandomForestModel/5'

In [69]:
y_pred_new = model.predict(X_test)

In [70]:
y_pred_new

array([0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 2, 2, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 1, 1,
       2])

## Note: *u only need to register after validating the model*